In [83]:
import pickle
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [84]:
data_path= "./data/UNLABELED_accounts_emojis_replaced.csv"
df = pd.read_csv(data_path)

print(df.head())

        username                                        description
0       WFLANews  Tampa Bay's latest news and information. Liste...
1   techtampabay  Tampa Bay's Tech Community Radically Connected...
2  FloridaIssues                                                NaN
3   LeChatNoire4  #VOTE BLUE 2022 :water wave::flag: United Stat...
4      SethPlatt  Creator Collector Cultivator Art Web3 ENS AI S...


In [85]:
model_path = "SVM_BOW_unweighted_enhanced_model_full.pickle"
with open(model_path, 'rb') as model_file:
    model = pickle.load(model_file)

In [86]:
lemmatizer = WordNetLemmatizer()

words_not_changed = ['media']
def preprocessing(row):
    row = str(row) 
    row = word_tokenize(row)  
    row = [lemmatizer.lemmatize(word) if word not in words_not_changed else word for word in row] 
    return str(row) 

In [87]:
df['description_lemmatized'] = df['description'].apply(preprocessing)
X = df['description_lemmatized']

In [88]:
print(X.head())

0    ['Tampa', 'Bay', "'s", 'latest', 'news', 'and'...
1    ['Tampa', 'Bay', "'s", 'Tech', 'Community', 'R...
2                                              ['nan']
3    ['#', 'VOTE', 'BLUE', '2022', ':', 'water', 'w...
4    ['Creator', 'Collector', 'Cultivator', 'Art', ...
Name: description_lemmatized, dtype: object


In [89]:
predicted_labels = model.predict(X)

In [90]:
print(predicted_labels)

['media' 'other' 'other' ... 'other' 'other' 'other']


In [91]:
unique_values, counts = np.unique(predicted_labels, return_counts=True)

value_counts = dict(zip(unique_values, counts))
value_counts_df = pd.DataFrame({'Value': unique_values, 'Count': counts})
print(value_counts_df)

     Value  Count
0     acad    503
1      gov     95
2    media   1305
3    other  20531
4  tourbiz     56


In [92]:
df['predicted_labels'] = predicted_labels

In [93]:
output_path = "./data/UNLABELED_accounts_with_predictions.csv"
df.to_csv(output_path, index=False)